# Text Generation for Email Creation

In [48]:
import json
import os
import sys

import boto3
import botocore

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime', 
    aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name='us-west-2'
)

In [49]:
# create the prompt
scenario_description = """
Task: Compose an email from Alice, Head of Client Relations, to the client "Mark Taylor" 
who expressed dissatisfaction with the technical support provided.
"""

In [50]:
request_body = json.dumps({
    "inputText": scenario_description, 
    "textGenerationConfig":{
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

In [51]:
modelId = 'amazon.titan-tg1-large'
accept = 'application/json'
contentType = 'application/json'
generatedText = "\n"

model_response = bedrock_runtime.invoke_model(body=request_body, modelId=modelId, accept=accept, contentType=contentType)
response_contents = json.loads(model_response.get('body').read())

generatedText = response_contents.get('results')[0].get('outputText')

In [52]:
email_content = generatedText[generatedText.index('\n')+1:]
print(email_content)


Dear Mark,

I hope this email finds you well. I am writing to express my sincere apologies for the inconvenience
caused to you due to the technical issue with our product. We understand the frustration and
disappointment this has caused, and we take your feedback very seriously.

I want to assure you that we are committed to resolving the problem as quickly as possible. Our
technical team is working tirelessly to identify the root cause and implement a solution. We
understand the importance of providing excellent customer support, and we deeply regret falling
short of your expectations.

In the meantime, we would like to offer you a temporary solution to minimize the impact on your
business. We understand that your time is valuable, and we want to ensure that you can continue to
operate efficiently. Our team will be available to assist you with any questions or concerns you may
have and to work with you to find the best possible solution.

Once again, please accept my sincere apologie

# Streaming Email Creation

In [53]:
output_data = []
streamed_response = bedrock_runtime.invoke_model_with_response_stream(body=request_body, modelId=modelId, accept=accept, contentType=contentType)
response_stream = streamed_response.get('body')

chunk_count = 1
if response_stream:
    for event in response_stream:
        data_chunk = event.get('chunk')
        if data_chunk:
            chunk_data = json.loads(data_chunk.get('bytes').decode())
            text_chunk = chunk_data['outputText']
            output_data.append(text_chunk)
            print(f'\t\t\x1b[31m**Segment {chunk_count}**\x1b[0m\n{text_chunk}\n')
            chunk_count += 1

		**Segment 1**
Subject: Apology for the Inconvenience

Dear Mark,

I hope this email finds you well. I am writing to express my sincere apologies for the inconvenience caused to you due to the technical issue with ou

		**Segment 2**
r product. We understand the frustration and disappointment this has caused, and we take your feedback very seriously.

I want to assure you that we are committed to resolving the problem as quickly as possible. Our technical team is working tirelessly to identify the root cause and implement a solution. We understand the importance of providing excellent customer su

		**Segment 3**
pport, and we deeply regret falling short of your expectations.

In the meantime, we would like to offer you a temporary solution to minimize the impact on your business. We understand that your time is valuable, and we want to ensure that you can continue to operate efficiently. Our team will be available to assist you with any questions or concer

		**Segment 4**
ns you may

In [54]:
print('\t\t\x1b[31m**FULL OUTPUT**\x1b[0m\n')
full_output_text = ''.join(output_data)
print(full_output_text)

		**FULL OUTPUT**

Subject: Apology for the Inconvenience

Dear Mark,

I hope this email finds you well. I am writing to express my sincere apologies for the inconvenience caused to you due to the technical issue with our product. We understand the frustration and disappointment this has caused, and we take your feedback very seriously.

I want to assure you that we are committed to resolving the problem as quickly as possible. Our technical team is working tirelessly to identify the root cause and implement a solution. We understand the importance of providing excellent customer support, and we deeply regret falling short of your expectations.

In the meantime, we would like to offer you a temporary solution to minimize the impact on your business. We understand that your time is valuable, and we want to ensure that you can continue to operate efficiently. Our team will be available to assist you with any questions or concerns you may have and to work with you to find the best possibl

# Summarization of Text

In [55]:
scenario_description = """
Please provide a summary of the following text. Do not add any information that is not mentioned in the text below.

<text>
Today, the city zoo unveiled its new exhibit, "Wild Wonders of Africa", which showcases a variety of African wildlife. The exhibit features animals such as giraffes, elephants, and lions in a naturalistic setting, offering a glimpse into the diverse ecosystems of Africa. The zoo has partnered with local schools to provide educational tours, aiming to foster a love for wildlife and conservation among the younger generation. Additionally, a portion of the proceeds from the exhibit will go towards supporting wildlife conservation efforts in Africa. The exhibit is expected to attract many visitors and raise awareness about the importance of preserving natural habitats.
</text>

"""

In [56]:
request_body = json.dumps({"inputText": scenario_description, 
                   "textGenerationConfig":{
                       "maxTokenCount":1000,
                       "stopSequences":[],
                       "temperature":0.5,
                       "topP":0.5
                   },
                  }) 

In [57]:
modelId = 'amazon.titan-tg1-large'
accept = 'application/json'
contentType = 'application/json'
generatedText = "\n"

model_response = bedrock_runtime.invoke_model(body=request_body, modelId=modelId, accept=accept, contentType=contentType)
response_contents = json.loads(model_response.get('body').read())

generatedText = response_contents.get('results')[0].get('outputText')


In [58]:
generatedText

'The "Wild Wonders of Africa" exhibit at the city zoo offers visitors a glimpse into the diverse ecosystems of Africa, featuring animals such as giraffes, elephants, and lions. The zoo has partnered with local schools to provide educational tours and a portion of the proceeds will go towards supporting wildlife conservation efforts in Africa. The exhibit is expected to attract many visitors and raise awareness about the importance of preserving natural habitats.'